In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
dataset=pd.read_csv('ShubmanGill-IPL.csv')

In [3]:
allviews=[]

In [4]:
mapping=defaultdict(set)
for index,row in dataset.iterrows():
    if row['p_bat'] not in mapping:
        mapping[row['bat']].add(row['p_bat'])

In [5]:
mapping

defaultdict(set,
            {'Shubman Gill': {1070173},
             'Andre Russell': {276298},
             'Eoin Morgan': {24598}})

In [6]:
def bowl_categoryfunc(bowl_type):
#     print(bowl_type)
    if bowl_type in ['RFM','RM','RMF','RF']:
        ans='RAP'
    elif bowl_type in ['LFM','LM','LMF','LF']:
        ans='LAP'
    elif bowl_type =='OB/LB':
        ans='RAS'
    elif bowl_type=='SLA/LWS':
        ans='LAS'
    elif bowl_type in ['LB','LBG']:
        ans='RWS'
    elif bowl_type=='SLA':
        ans='SLA'
    elif bowl_type=='OB':
        ans='RAO'
    elif bowl_type=='LWS':
        ans='LWS'
    return ans

In [7]:
calculate={
#     'inns_number':'first',
#     'team':'first',
    'runs':'sum',
    'balls':'sum',
    'out':'sum',
    'bowler_dismissed?':'sum',
    'dots':'sum',
    '1s':'sum',
    '2s':'sum',
    '3s':'sum',
    '4s':'sum',
    '5s':'sum',
    '6s':'sum',
#     'opposition':'first',
    'control':'sum'
#     'ground':'first',
#     'date':'first'
}

In [8]:
fullview={
    'inns_number':'first',
    'team':'first',
    'runs':'sum',
    'balls':'sum',
    'out':'sum',
    'bowler_dismissed?':'sum',
    'dots':'sum',
    '1s':'sum',
    '2s':'sum',
    '3s':'sum',
    '4s':'sum',
    '5s':'sum',
    '6s':'sum',
    'opposition':'first',
    'control':'sum',
    'ground':'first'
}

In [9]:
dataset.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [10]:
dataset

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,bat_out,wagonX,wagonY,wagonZone,line,length,shot,control,predscore,wprob
0,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,1,2.01,...,True,202,219,4,OUTSIDE_OFFSTUMP,FULL,PUSH,1.0,165,46.47
1,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,4,2.04,...,True,154,224,5,OUTSIDE_OFFSTUMP,FULL,ON_DRIVE,1.0,162,43.17
2,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,5,2.05,...,True,221,193,3,ON_THE_STUMPS,GOOD_LENGTH,FLICK,1.0,160,41.01
3,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,6,2.06,...,True,136,209,5,OUTSIDE_OFFSTUMP,FULL,COVER_DRIVE,1.0,159,39.94
4,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,1,5.01,...,True,324,135,2,ON_THE_STUMPS,GOOD_LENGTH,FLICK,1.0,166,47.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,2,13.02,...,True,325,49,2,WIDE_OUTSIDE_OFFSTUMP,SHORT,HOOK,1.0,255,96.93
2433,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,3,13.03,...,True,137,357,5,OUTSIDE_OFFSTUMP,FULL,ON_DRIVE,0.0,259,97.41
2434,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,4,13.04,...,True,174,205,5,ON_THE_STUMPS,YORKER,DEFENDED,1.0,257,97.18
2435,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,5,13.05,...,True,23,302,6,OUTSIDE_OFFSTUMP,FULL,COVER_DRIVE,1.0,260,97.52


In [11]:
dataset.drop_duplicates(inplace=True)

In [12]:
dataset.dtypes

p_match             int64
inns                int64
bat                object
p_bat               int64
team_bat           object
bowl               object
p_bowl              int64
team_bowl          object
ball                int64
ball_id           float64
outcome            object
score               int64
out                  bool
dismissal          object
p_out               int64
over                int64
noball              int64
wide                int64
byes                int64
legbyes             int64
cur_bat_runs        int64
cur_bat_bf          int64
cur_bowl_ovr      float64
cur_bowl_wkts       int64
cur_bowl_runs       int64
inns_runs           int64
inns_wkts           int64
inns_balls          int64
inns_runs_rem     float64
inns_balls_rem      int64
inns_rr           float64
inns_rrr          float64
target            float64
max_balls           int64
date               object
year                int64
ground             object
country            object
winner      

In [13]:
dataset['batruns']=dataset['batruns'].apply(lambda x: int(x))
dataset['ballfaced']=dataset['ballfaced'].apply(lambda x: int(x))
dataset['bowl_category']=dataset['bowl_style'].apply(bowl_categoryfunc)
# dataset['runout']=False

In [14]:
dataset.dtypes

p_match             int64
inns                int64
bat                object
p_bat               int64
team_bat           object
bowl               object
p_bowl              int64
team_bowl          object
ball                int64
ball_id           float64
outcome            object
score               int64
out                  bool
dismissal          object
p_out               int64
over                int64
noball              int64
wide                int64
byes                int64
legbyes             int64
cur_bat_runs        int64
cur_bat_bf          int64
cur_bowl_ovr      float64
cur_bowl_wkts       int64
cur_bowl_runs       int64
inns_runs           int64
inns_wkts           int64
inns_balls          int64
inns_runs_rem     float64
inns_balls_rem      int64
inns_rr           float64
inns_rrr          float64
target            float64
max_balls           int64
date               object
year                int64
ground             object
country            object
winner      

In [15]:
def phasecalc(m):
    if 1<=m<=6:
        return 'PP'
    elif 7<=m<=16:
        return 'Middle'
    elif 17<=m<=20:
        return 'Slog'

dataset['phase']=dataset['over'].apply(phasecalc)

In [16]:
modify=dataset[dataset['bat']!='Shubman Gill']
modify

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,wagonY,wagonZone,line,length,shot,control,predscore,wprob,bowl_category,phase
455,1136586,2,Andre Russell,276298,Kolkata Knight Riders,Amit Mishra,31107,Delhi Capitals,3,15.03,...,138,7,OUTSIDE_OFFSTUMP,GOOD_LENGTH,CUT_SHOT,1.0,181,0.21,RWS,Middle
1021,1216540,2,Eoin Morgan,24598,Kolkata Knight Riders,Washington Sundar,719715,Royal Challengers Bangalore,3,9.03,...,167,3,OUTSIDE_OFFSTUMP,FULL,DEFENDED,0.0,154,3.29,RAO,Middle


In [17]:
dataset.loc[modify.index,'bat']='Shubman Gill'
dataset.loc[modify.index,'batruns']=0
dataset.loc[modify.index,'ballfaced']=0
dataset.loc[modify.index,'bat_hand']='RHB'
dataset.loc[modify.index,'control']=0

In [18]:
runoutdis=dataset[dataset['dismissal']=='run out']

In [19]:
for index,row in runoutdis.iterrows():
    if row['p_out']==int(str(mapping[row['bat']])[1:-1]):
        runoutdis.loc[index, 'out'] = True
    else:
        runoutdis.loc[index, 'out'] = False

In [20]:
# runoutdis['runout']=runoutdis['out']
# runoutdis['out']=False
dataset.update(runoutdis)

In [21]:
reqcols=['inns','bat','team_bat','bowl','team_bowl','ball','ball_id','out','dismissal','over','phase','noball','wide','date','year','ground','country','bat_hand','bowl_style','bowl_kind','bowl_category','batruns','ballfaced','line','length','shot','control']
dataset=dataset[reqcols]

In [22]:
dataset.rename(columns={'inns':'inns_number','batruns':'runs','ballfaced':'balls','team_bowl':'opposition','team_bat':'team'},inplace=True)

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\228903517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.rename(columns={'inns':'inns_number','batruns':'runs','ballfaced':'balls','team_bowl':'opposition','team_bat':'team'},inplace=True)


In [23]:
dataset['inns']=1

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\3667509679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['inns']=1


In [24]:
dataset.duplicated().sum()

0

In [25]:
dataset.isnull().sum()

inns_number         0
bat                 0
team                0
bowl                0
opposition          0
ball                0
ball_id             0
out                 0
dismissal        2346
over                0
phase               0
noball              0
wide                0
date                0
year                0
ground              0
country             0
bat_hand            0
bowl_style          0
bowl_kind           0
bowl_category       0
runs                0
balls               0
line                7
length              7
shot                2
control            39
inns                0
dtype: int64

In [26]:
dataset['1s']=dataset['runs'].apply(lambda x: 1 if x==1 else 0)
dataset['2s']=dataset['runs'].apply(lambda x: 1 if x==2 else 0)
dataset['3s']=dataset['runs'].apply(lambda x: 1 if x==3 else 0)
dataset['4s']=dataset['runs'].apply(lambda x: 1 if x==4 else 0)
dataset['5s']=dataset['runs'].apply(lambda x: 1 if x==5 else 0)
dataset['6s']=dataset['runs'].apply(lambda x: 1 if x==6 else 0)

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\1259893229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['1s']=dataset['runs'].apply(lambda x: 1 if x==1 else 0)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\1259893229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['2s']=dataset['runs'].apply(lambda x: 1 if x==2 else 0)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\1259893229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [27]:
dataset

,inns_number,bat,team,bowl,opposition,ball,ball_id,out,dismissal,over,...,length,shot,control,inns,1s,2s,3s,4s,5s,6s
0,1.0,Shubman Gill,Kolkata Knight Riders,Lasith Malinga,Mumbai Indians,1.0,2.01,False,NaN,3.0,...,FULL,PUSH,1.0,1,1,0,0,0,0,0
1,1.0,Shubman Gill,Kolkata Knight Riders,Lasith Malinga,Mumbai Indians,4.0,2.04,False,NaN,3.0,...,FULL,ON_DRIVE,1.0,1,0,0,0,0,0,0
2,1.0,Shubman Gill,Kolkata Knight Riders,Lasith Malinga,Mumbai Indians,5.0,2.05,False,NaN,3.0,...,GOOD_LENGTH,FLICK,1.0,1,0,0,0,0,0,0
3,1.0,Shubman Gill,Kolkata Knight Riders,Lasith Malinga,Mumbai Indians,6.0,2.06,False,NaN,3.0,...,FULL,COVER_DRIVE,1.0,1,1,0,0,0,0,0
4,1.0,Shubman Gill,Kolkata Knight Riders,Lasith Malinga,Mumbai Indians,1.0,5.01,False,NaN,6.0,...,GOOD_LENGTH,FLICK,1.0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,1.0,Shubman Gill,Gujarat Titans,Daryl Mitchell,Chennai Super Kings,2.0,13.02,False,NaN,14.0,...,SHORT,HOOK,1.0,1,0,0,0,0,0,1
2433,1.0,Shubman Gill,Gujarat Titans,Daryl Mitchell,Chennai Super Kings,3.0,13.03,False,NaN,14.0,...,FULL,ON_DRIVE,0.0,1,0,0,0,0,0,1
2434,1.0,Shubman Gill,Gujarat Titans,Daryl Mitchell,Chennai Super Kings,4.0,13.04,False,NaN,14.0,...,YORKER,DEFENDED,1.0,1,0,0,0,0,0,0
2435,1.0,Shubman Gill,Gujarat Titans,Daryl Mitchell,Chennai Super Kings,5.0,13.05,False,NaN,14.0,...,FULL,COVER_DRIVE,1.0,1,0,0,0,0,0,1


In [28]:
dataset['out']=dataset['out'].astype(int)
dataset['dismissal'].value_counts()

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\34766640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['out']=dataset['out'].astype(int)


caught               57
run out              11
bowled               10
leg before wicket     6
stumped               4
Name: dismissal, dtype: int64

In [29]:
def filterview(x):
    x['avg']=x['runs']/x['out']
    x['avg']=round(x['avg'],2)
    x['SR']=x['runs']*100/x['balls']
    x['SR']=round(x['SR'],2)
    x['Dots%']=x['dots']*100/x['balls']
    x['Dots%']=round(x['Dots%'],2)
    x['Control%']=x['control']*100/x['balls']
    x['Control%']=round(x['Control%'],2)
    x.drop('control',axis=1,inplace=True)
    return x

In [30]:
dataset['bowler_dismissed?']=dataset['dismissal'].apply(lambda x: 1 if x in ['caught','bowled','leg before wicket','stumped','hit wicket'] else 0)

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\731582508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['bowler_dismissed?']=dataset['dismissal'].apply(lambda x: 1 if x in ['caught','bowled','leg before wicket','stumped','hit wicket'] else 0)


In [31]:
dataset['dots']=0
dataset.loc[(dataset['runs']==0) & (dataset['balls'])==1,'dots']=1

C:\Users\Vijay\AppData\Local\Temp\ipykernel_19104\4249748624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['dots']=0


In [32]:
profile=filterview(dataset.groupby(['bat']).aggregate(calculate)).reset_index()
profile

,bat,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,Shubman Gill,3216.0,2370.0,85,77,761,1012,182,10,310,0,95,37.84,135.7,32.11,85.49


In [33]:
yearwise=filterview(dataset.groupby(['year']).aggregate(calculate)).reset_index()
yearwise

,year,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,2018.0,203.0,139.0,6,5,37,65,10,0,22,0,5,33.83,146.04,26.62,80.58
1,2019.0,296.0,238.0,9,8,76,112,17,2,21,0,10,32.89,124.37,31.93,86.13
2,2020.0,440.0,373.0,13,11,134,162,24,0,44,0,9,33.85,117.96,35.92,86.86
3,2021.0,478.0,402.0,17,15,162,152,24,2,50,0,12,28.12,118.91,40.30,83.33
4,2022.0,483.0,365.0,14,12,117,159,27,0,51,0,11,34.50,132.33,32.05,84.93
5,2023.0,890.0,564.0,15,15,150,244,48,4,85,0,33,59.33,157.80,26.60,87.77
6,2024.0,426.0,289.0,11,11,85,118,32,2,37,0,15,38.73,147.40,29.41,84.78


In [34]:
inningsbyinnings=(dataset.groupby(['date']).aggregate(fullview)).reset_index()
inningsbyinnings

,date,inns_number,team,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,opposition,control,ground
0,2018-04-14,1.0,Kolkata Knight Riders,3.0,9.0,1,1,6,3,0,0,0,0,0,Sunrisers Hyderabad,6.0,"Eden Gardens, Kolkata"
1,2018-04-16,1.0,Kolkata Knight Riders,6.0,5.0,1,1,2,2,0,0,1,0,0,Delhi Capitals,3.0,"Eden Gardens, Kolkata"
2,2018-04-21,1.0,Kolkata Knight Riders,14.0,8.0,0,0,1,4,1,0,2,0,0,Punjab Kings,8.0,"Eden Gardens, Kolkata"
3,2018-04-27,2.0,Kolkata Knight Riders,37.0,29.0,1,0,8,15,2,0,3,0,1,Delhi Capitals,21.0,"Feroz Shah Kotla, Delhi"
4,2018-04-29,2.0,Kolkata Knight Riders,5.0,2.0,0,0,0,1,0,0,1,0,0,Royal Challengers Bangalore,2.0,"M Chinnaswamy Stadium, Bengaluru"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2024-04-21,2.0,Gujarat Titans,35.0,29.0,1,1,12,9,3,0,5,0,0,Punjab Kings,25.0,Maharaja Yadavindra Singh International Cricke...
95,2024-04-24,2.0,Gujarat Titans,6.0,5.0,1,1,2,2,0,0,1,0,0,Delhi Capitals,2.0,"Arun Jaitley Stadium, Delhi"
96,2024-04-28,1.0,Gujarat Titans,16.0,19.0,1,1,8,8,2,0,1,0,0,Royal Challengers Bengaluru,17.0,"Narendra Modi Stadium, Ahmedabad"
97,2024-05-04,1.0,Gujarat Titans,2.0,7.0,1,1,5,2,0,0,0,0,0,Royal Challengers Bengaluru,5.0,"M Chinnaswamy Stadium, Bengaluru"


In [35]:
inningsbyinnings.groupby('inns_number').aggregate(calculate).reset_index()
filterview(inningsbyinnings)

,date,inns_number,team,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,opposition,ground,avg,SR,Dots%,Control%
0,2018-04-14,1.0,Kolkata Knight Riders,3.0,9.0,1,1,6,3,0,0,0,0,0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",3.0,33.33,66.67,66.67
1,2018-04-16,1.0,Kolkata Knight Riders,6.0,5.0,1,1,2,2,0,0,1,0,0,Delhi Capitals,"Eden Gardens, Kolkata",6.0,120.00,40.00,60.00
2,2018-04-21,1.0,Kolkata Knight Riders,14.0,8.0,0,0,1,4,1,0,2,0,0,Punjab Kings,"Eden Gardens, Kolkata",inf,175.00,12.50,100.00
3,2018-04-27,2.0,Kolkata Knight Riders,37.0,29.0,1,0,8,15,2,0,3,0,1,Delhi Capitals,"Feroz Shah Kotla, Delhi",37.0,127.59,27.59,72.41
4,2018-04-29,2.0,Kolkata Knight Riders,5.0,2.0,0,0,0,1,0,0,1,0,0,Royal Challengers Bangalore,"M Chinnaswamy Stadium, Bengaluru",inf,250.00,0.00,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2024-04-21,2.0,Gujarat Titans,35.0,29.0,1,1,12,9,3,0,5,0,0,Punjab Kings,Maharaja Yadavindra Singh International Cricke...,35.0,120.69,41.38,86.21
95,2024-04-24,2.0,Gujarat Titans,6.0,5.0,1,1,2,2,0,0,1,0,0,Delhi Capitals,"Arun Jaitley Stadium, Delhi",6.0,120.00,40.00,40.00
96,2024-04-28,1.0,Gujarat Titans,16.0,19.0,1,1,8,8,2,0,1,0,0,Royal Challengers Bengaluru,"Narendra Modi Stadium, Ahmedabad",16.0,84.21,42.11,89.47
97,2024-05-04,1.0,Gujarat Titans,2.0,7.0,1,1,5,2,0,0,0,0,0,Royal Challengers Bengaluru,"M Chinnaswamy Stadium, Bengaluru",2.0,28.57,71.43,71.43


In [36]:
# inningsbyinnings['SR']=inningsbyinnings['runs']*100/inningsbyinnings['balls']
# inningsbyinnings['SR']=inningsbyinnings['SR'].round(2)
# inningsbyinnings['Control%']=inningsbyinnings['control']*100/inningsbyinnings['balls']
# inningsbyinnings['Control%']=inningsbyinnings['Control%'].round(2)
# inningsbyinnings['Dot%']=inningsbyinnings['dots']*100/inningsbyinnings['balls']
# inningsbyinnings['Dot%']=inningsbyinnings['Dot%'].round(2)

In [37]:
# inningsbyinnings.drop(columns=['dot'],axis=1,inplace=True)
# inningsbyinnings.drop(columns=['control'],axis=1,inplace=True)
# inningsbyinnings

In [38]:
h2h=filterview(dataset.groupby(['bowl','bowl_category']).aggregate(calculate).sort_values(by='runs',ascending=False)).reset_index()

In [39]:
h2h

,bowl,bowl_category,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,Ravichandran Ashwin,RAO,111.0,69.0,1,0,14,35,7,0,8,0,5,111.0,160.87,20.29,92.75
1,Ravindra Jadeja,SLA,79.0,56.0,1,1,14,25,8,0,8,0,1,79.0,141.07,25.00,87.50
2,Kagiso Rabada,RAP,78.0,59.0,2,2,21,24,4,0,7,0,3,39.0,132.20,35.59,81.36
3,Deepak Chahar,RAP,78.0,55.0,5,4,19,22,2,0,10,0,2,15.6,141.82,34.55,78.18
4,Rahul Chahar,RWS,75.0,42.0,2,2,10,15,8,0,5,0,4,37.5,178.57,23.81,88.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Arjun Tendulkar,LAP,1.0,1.0,0,0,0,1,0,0,0,0,0,inf,100.00,0.00,0.00
144,Tom Curran,RAP,1.0,1.0,0,0,0,1,0,0,0,0,0,inf,100.00,0.00,200.00
145,Fabian Allen,SLA,1.0,1.0,0,0,0,1,0,0,0,0,0,inf,100.00,0.00,100.00
146,Carlos Brathwaite,RAP,0.0,1.0,1,1,1,0,0,0,0,0,0,0.0,0.00,100.00,0.00


In [40]:
cumulativeprogression=dataset.groupby(by='date').aggregate(calculate).reset_index()

In [41]:
cumulativeprogression['inns']=cumulativeprogression.index + 1

In [42]:
cumulativeprogression

,date,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,control,inns
0,2018-04-14,3.0,9.0,1,1,6,3,0,0,0,0,0,6.0,1
1,2018-04-16,6.0,5.0,1,1,2,2,0,0,1,0,0,3.0,2
2,2018-04-21,14.0,8.0,0,0,1,4,1,0,2,0,0,8.0,3
3,2018-04-27,37.0,29.0,1,0,8,15,2,0,3,0,1,21.0,4
4,2018-04-29,5.0,2.0,0,0,0,1,0,0,1,0,0,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2024-04-21,35.0,29.0,1,1,12,9,3,0,5,0,0,25.0,95
95,2024-04-24,6.0,5.0,1,1,2,2,0,0,1,0,0,2.0,96
96,2024-04-28,16.0,19.0,1,1,8,8,2,0,1,0,0,17.0,97
97,2024-05-04,2.0,7.0,1,1,5,2,0,0,0,0,0,5.0,98


In [43]:
desired_columns = ['inns', 'runs', 'balls', 
                   'dots', '1s', '2s', '3s', '4s', '5s', '6s', 
                   'control', 'out', 'wide', 'noball','year', 'date']
correctview=['runs','balls','avg','SR','dots','1s','2s','3s','4s','5s','6s','Dots%','Control%']
cumulativeprogression=cumulativeprogression.reindex(columns=desired_columns)

In [44]:
cumulativeprogression=cumulativeprogression.assign(cumulative_runs=cumulativeprogression['runs'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_balls=cumulativeprogression['balls'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_dots=cumulativeprogression['dots'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_1s=cumulativeprogression['1s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_2s=cumulativeprogression['2s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_3s=cumulativeprogression['3s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_4s=cumulativeprogression['4s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_5s=cumulativeprogression['5s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_6s=cumulativeprogression['6s'].cumsum())
cumulativeprogression=cumulativeprogression.assign(cumulative_control=cumulativeprogression['control'].cumsum())

In [45]:
cumulativeprogression

,inns,runs,balls,dots,1s,2s,3s,4s,5s,6s,...,cumulative_runs,cumulative_balls,cumulative_dots,cumulative_1s,cumulative_2s,cumulative_3s,cumulative_4s,cumulative_5s,cumulative_6s,cumulative_control
0,1,3.0,9.0,6,3,0,0,0,0,0,...,3.0,9.0,6,3,0,0,0,0,0,6.0
1,2,6.0,5.0,2,2,0,0,1,0,0,...,9.0,14.0,8,5,0,0,1,0,0,9.0
2,3,14.0,8.0,1,4,1,0,2,0,0,...,23.0,22.0,9,9,1,0,3,0,0,17.0
3,4,37.0,29.0,8,15,2,0,3,0,1,...,60.0,51.0,17,24,3,0,6,0,1,38.0
4,5,5.0,2.0,0,1,0,0,1,0,0,...,65.0,53.0,17,25,3,0,7,0,1,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,35.0,29.0,12,9,3,0,5,0,0,...,3088.0,2284.0,735,974,177,10,299,0,89,1957.0
95,96,6.0,5.0,2,2,0,0,1,0,0,...,3094.0,2289.0,737,976,177,10,300,0,89,1959.0
96,97,16.0,19.0,8,8,2,0,1,0,0,...,3110.0,2308.0,745,984,179,10,301,0,89,1976.0
97,98,2.0,7.0,5,2,0,0,0,0,0,...,3112.0,2315.0,750,986,179,10,301,0,89,1981.0


In [46]:
dataset.columns

Index(['inns_number', 'bat', 'team', 'bowl', 'opposition', 'ball', 'ball_id',
       'out', 'dismissal', 'over', 'phase', 'noball', 'wide', 'date', 'year',
       'ground', 'country', 'bat_hand', 'bowl_style', 'bowl_kind',
       'bowl_category', 'runs', 'balls', 'line', 'length', 'shot', 'control',
       'inns', '1s', '2s', '3s', '4s', '5s', '6s', 'bowler_dismissed?',
       'dots'],
      dtype='object')

In [47]:
bowlstyle=dataset.groupby(by=['bowl_category']).aggregate(calculate).reset_index()
bowlstyle=filterview(bowlstyle)

In [48]:
bowlstyle

,bowl_category,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,LAP,525.0,402.0,7,6,129,175,27,2,62,0,7,75.00,130.60,32.09,85.32
1,LAS,34.0,22.0,1,1,6,10,2,0,2,0,2,34.00,154.55,27.27,86.36
2,LWS,20.0,11.0,0,0,2,4,3,0,1,0,1,inf,181.82,18.18,100.00
3,RAO,258.0,185.0,5,3,45,92,21,0,19,0,8,51.60,139.46,24.32,90.27
4,RAP,1455.0,1086.0,50,47,388,429,63,6,159,0,41,29.10,133.98,35.73,82.41
5,RAS,22.0,15.0,1,1,6,4,1,0,4,0,0,22.00,146.67,40.00,80.00
6,RWS,510.0,359.0,18,16,92,171,38,1,41,0,16,28.33,142.06,25.63,88.02
7,SLA,392.0,290.0,3,3,93,127,27,1,22,0,20,130.67,135.17,32.07,90.69


In [49]:
line_length=dataset.groupby(['bowl_kind','line','length']).aggregate(calculate).sort_values(by='runs',ascending=False).reset_index()
line_length=filterview(line_length)

In [50]:
line_length

,bowl_kind,line,length,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,pace bowler,OUTSIDE_OFFSTUMP,SHORT_OF_A_GOOD_LENGTH,435.0,310.0,9,9,104,118,22,1,60,0,5,48.33,140.32,33.55,80.00
1,spin bowler,OUTSIDE_OFFSTUMP,GOOD_LENGTH,358.0,253.0,10,8,72,114,25,0,29,0,13,35.80,141.50,28.46,87.75
2,pace bowler,OUTSIDE_OFFSTUMP,GOOD_LENGTH,306.0,297.0,9,8,133,114,10,2,31,0,7,34.00,103.03,44.78,83.50
3,spin bowler,ON_THE_STUMPS,GOOD_LENGTH,290.0,243.0,7,6,75,118,26,0,12,0,12,41.43,119.34,30.86,90.12
4,pace bowler,ON_THE_STUMPS,GOOD_LENGTH,246.0,219.0,6,6,78,99,16,1,19,0,6,41.00,112.33,35.62,90.87
5,pace bowler,OUTSIDE_OFFSTUMP,FULL,216.0,155.0,8,8,56,60,10,0,19,0,10,27.00,139.35,36.13,79.35
6,pace bowler,ON_THE_STUMPS,FULL,162.0,110.0,8,7,29,50,8,0,21,0,2,20.25,147.27,26.36,84.55
7,pace bowler,OUTSIDE_OFFSTUMP,SHORT,159.0,75.0,3,3,17,26,5,1,18,0,8,53.00,212.00,22.67,76.00
8,spin bowler,OUTSIDE_OFFSTUMP,FULL,148.0,98.0,6,5,29,44,5,0,13,0,7,24.67,151.02,29.59,90.82
9,spin bowler,ON_THE_STUMPS,FULL,136.0,94.0,4,4,27,44,8,0,7,0,8,34.00,144.68,28.72,88.30


In [51]:
groundwise=dataset.groupby('ground').aggregate(calculate).sort_values(by='runs',ascending=False).reset_index()

In [52]:
groundwise=filterview(groundwise)
groundwise

,ground,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,"Narendra Modi Stadium, Ahmedabad",953.0,598.0,15,15,149,265,58,6,83,0,37,63.53,159.36,24.92,86.45
1,"Eden Gardens, Kolkata",419.0,290.0,11,10,85,131,16,2,43,0,13,38.09,144.48,29.31,84.48
2,"Zayed Cricket Stadium, Abu Dhabi",282.0,254.0,10,8,102,98,20,0,30,0,4,28.20,111.02,40.16,88.98
3,Sharjah Cricket Stadium,280.0,233.0,7,6,77,114,11,2,18,0,11,40.00,120.17,33.05,81.97
4,Dubai International Cricket Stadium,224.0,176.0,6,6,65,66,12,0,32,0,1,37.33,127.27,36.93,82.95
5,"Brabourne Stadium, Mumbai",179.0,123.0,3,3,36,51,12,0,20,0,4,59.67,145.53,29.27,86.99
6,"Maharashtra Cricket Association Stadium, Pune",147.0,96.0,2,2,16,55,8,0,13,0,4,73.50,153.12,16.67,88.54
7,"Punjab Cricket Association IS Bindra Stadium, ...",132.0,98.0,1,1,26,48,9,0,12,0,3,132.00,134.69,26.53,93.88
8,"MA Chidambaram Stadium, Chepauk, Chennai",128.0,101.0,6,6,45,32,6,0,12,0,6,21.33,126.73,44.55,89.11
9,"Sawai Mansingh Stadium, Jaipur",114.0,89.0,2,2,30,36,9,0,12,0,2,57.00,128.09,33.71,89.89


In [53]:
forteam=dataset.groupby('team').aggregate(calculate).reset_index()
forteam=filterview(forteam)

In [54]:
forteam

,team,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,Gujarat Titans,1799.0,1218.0,40,38,352,521,107,6,173,0,59,44.98,147.7,28.9,86.21
1,Kolkata Knight Riders,1417.0,1152.0,45,39,409,491,75,4,137,0,36,31.49,123.0,35.5,84.72


In [55]:
line=dataset.groupby(['bowl_kind','line']).aggregate(calculate).sort_values(by='runs',ascending=False).reset_index()
line=filterview(line)

In [56]:
line

,bowl_kind,line,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,pace bowler,OUTSIDE_OFFSTUMP,1168.0,876.0,29,28,320,337,51,5,132,0,31,40.28,133.33,36.53,81.05
1,spin bowler,OUTSIDE_OFFSTUMP,638.0,447.0,16,13,125,201,44,1,55,0,21,39.88,142.73,27.96,89.71
2,pace bowler,ON_THE_STUMPS,616.0,483.0,20,17,152,222,32,2,63,0,12,30.80,127.54,31.47,88.20
3,spin bowler,ON_THE_STUMPS,522.0,388.0,11,10,110,183,43,1,28,0,23,47.45,134.54,28.35,89.69
4,pace bowler,DOWN_LEG,104.0,61.0,4,4,18,23,2,1,14,0,3,26.00,170.49,29.51,85.25
5,pace bowler,WIDE_OUTSIDE_OFFSTUMP,85.0,61.0,4,4,24,19,5,0,11,0,2,21.25,139.34,39.34,72.13
6,spin bowler,DOWN_LEG,38.0,25.0,0,0,5,14,2,0,2,0,2,inf,152.00,20.00,84.00
7,spin bowler,WIDE_OUTSIDE_OFFSTUMP,38.0,22.0,1,1,4,10,3,0,4,0,1,38.00,172.73,18.18,81.82
8,pace bowler,WIDE_DOWN_LEG,0.0,0.0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,inf
9,spin bowler,WIDE_DOWN_LEG,0.0,0.0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [57]:
length=dataset.groupby(['length','bowl_kind']).aggregate(calculate).reset_index()
length=filterview(length)

In [58]:
length

,length,bowl_kind,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,FULL,pace bowler,425.0,295.0,19,18,95,119,22,0,46,0,13,22.37,144.07,32.20,80.34
1,FULL,spin bowler,321.0,214.0,11,10,62,97,15,0,23,0,17,29.18,150.00,28.97,89.25
2,FULL_TOSS,pace bowler,73.0,52.0,3,3,12,26,5,1,7,0,1,24.33,140.38,23.08,88.46
3,FULL_TOSS,spin bowler,16.0,4.0,0,0,1,0,0,0,1,0,2,inf,400.00,25.00,100.00
4,GOOD_LENGTH,pace bowler,588.0,543.0,17,16,220,225,27,3,54,0,14,34.59,108.29,40.52,86.74
5,GOOD_LENGTH,spin bowler,667.0,509.0,17,14,149,241,51,0,42,0,26,39.24,131.04,29.27,88.80
6,SHORT,pace bowler,236.0,106.0,4,4,23,37,6,1,25,0,14,59.00,222.64,21.70,80.19
7,SHORT,spin bowler,13.0,7.0,0,0,0,5,0,0,2,0,0,inf,185.71,0.00,100.00
8,SHORT_OF_A_GOOD_LENGTH,pace bowler,613.0,442.0,13,12,145,176,27,3,86,0,5,47.15,138.69,32.81,81.00
9,SHORT_OF_A_GOOD_LENGTH,spin bowler,219.0,146.0,0,0,30,65,26,2,21,0,2,inf,150.00,20.55,91.10


In [59]:
shottype=filterview(dataset.groupby(['bowl_kind','shot']).aggregate(calculate).sort_values(by='runs',ascending=False).reset_index())

In [60]:
shottype

,bowl_kind,shot,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,pace bowler,FLICK,444.0,320.0,12,11,70,167,32,3,42,0,6,37.00,138.75,21.88,86.25
1,pace bowler,PULL,373.0,181.0,7,7,28,78,17,1,42,0,15,53.29,206.08,15.47,77.90
2,pace bowler,ON_DRIVE,314.0,229.0,5,5,68,114,7,0,27,0,13,62.80,137.12,29.69,87.77
3,pace bowler,CUT_SHOT,242.0,146.0,6,6,43,52,8,0,42,0,1,40.33,165.75,29.45,69.86
4,spin bowler,FLICK,225.0,196.0,3,2,32,123,34,0,4,0,3,75.00,114.80,16.33,93.37
5,spin bowler,ON_DRIVE,222.0,175.0,9,9,23,128,10,0,5,0,9,24.67,126.86,13.14,88.57
6,pace bowler,COVER_DRIVE,211.0,154.0,6,6,58,45,18,4,28,0,1,35.17,137.01,37.66,83.12
7,spin bowler,COVER_DRIVE,186.0,113.0,2,2,33,40,12,0,23,0,5,93.00,164.60,29.20,97.35
8,spin bowler,CUT_SHOT,130.0,97.0,3,1,28,33,23,1,12,0,0,43.33,134.02,28.87,87.63
9,spin bowler,PULL,99.0,48.0,0,0,3,26,2,1,15,0,1,inf,206.25,6.25,95.83


In [61]:
phasewise=filterview(dataset.groupby(['phase']).aggregate(calculate)).reset_index()
phasewise

,phase,runs,balls,out,bowler_dismissed?,dots,1s,2s,3s,4s,5s,6s,avg,SR,Dots%,Control%
0,Middle,1516.0,1081.0,40,36,267,534,119,4,105,0,52,37.90,140.24,24.70,86.40
1,PP,1432.0,1113.0,31,28,454,387,49,5,188,0,30,46.19,128.66,40.79,85.27
2,Slog,268.0,176.0,14,13,40,91,14,1,17,0,13,19.14,152.27,22.73,81.25
